# Phase III Project Technical Notebook

#### Authors: Kyle Dufrane and Brad Horn

### Overview

#### This project analyzes the Tanzanian Water Wells datasets released by the Tanzanian Government. The dataset includes 59,400 rows each repressenting a unique well within the Tanzanian Governments realm. Our targets are broken down into three categories:

* Functional
* Non Functional
* Functional Needs Repair

#### We will attempt to predict the status of the wells condition through utilizing Exploratory Data Analysis (EDA) and building classification models tuned to the parameters that will have the largest impact on our predictive ability. 


### Business Understanding

#### Flatiron LLC has recently been awarded a contract to maintain wells in Tanzania. They're looking for a system to help develop preventative maintenance schedules by predicting pump failures and replacement schedules to better serve their client. Flatiron LLC would like key insights on:

* Regional impact on wells
* Area's with low water quantity
* Negatively impacting factors on wells 


### Data Understanding

#### This dataset comes with two applicable files training_set_labels and training_set_values. During our EDA we will join these tables together to give us one file to work with. The values dataset has 39 total columns and contains all of our predicitve features. Below is a description of each column. 

* amount_tsh : Total static head (amount water available to waterpoint)
* date_recorded : The date the row was entered
* funder : Who funded the well
* gps_height : Altitude of the well
* installer : Organization that installed the well
* longitude : GPS coordinate
* latitude : GPS coordinate
* wpt_name : Name of the waterpoint if there is one
* num_private :Private use or not
* basin : Geographic water basin
* subvillage : Geographic location
* region : Geographic location
* region_code : Geographic location (coded)
* district_code : Geographic location (coded)
* lga : Geographic location
* ward : Geographic location
* population : Population around the well
* public_meeting : True/False
* recorded_by : Group entering this row of data
* scheme_management : Who operates the waterpoint
* scheme_name : Who operates the waterpoint
* permit : If the waterpoint is permitted
* construction_year : Year the waterpoint was constructed
* extraction_type : The kind of extraction the waterpoint uses
* extraction_type_group : The kind of extraction the waterpoint uses
* extraction_type_class : The kind of extraction the waterpoint uses
* management : How the waterpoint is managed
* management_group : How the waterpoint is managed
* payment : What the water costs
* payment_type : What the water costs
* water_quality : The quality of the water
* quality_group : The quality of the water
* quantity : The quantity of water
* quantity_group : The quantity of water
* source : The source of the water
* source_type : The source of the water
* source_class : The source of the water
* waterpoint_type : The kind of waterpoint
* waterpoint_type_group : The kind of waterpoint

#### To start we will import all of our needed libraries and dive into our datasets.

In [1]:
# Import needed libraries

# Import libaries needed for EDA and visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Import Pickle to saved files giving us the ability to only run each model once.
import pickle

# Import needed SKLearn libraries for modeling, imputing, and pipelines
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, plot_confusion_matrix
from xgboost import XGBClassifier

# Import YellowBrick to easily plot ROC AUC curves on multiclass algorithms
from yellowbrick.classifier import ROCAUC

pd.set_option('display.max_columns', 999)

### Import and basic interpretation on our datasets

In [2]:
# Import training labels CSV
df_training_labels = pd.read_csv('data/Training_set_labels.csv')

In [3]:
df_training_labels.shape

(59400, 2)

In [4]:
df_training_labels.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [5]:
df_training_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


In [6]:
df_training_labels['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

#### Revewing the above information we do not have any nulls in our this dataset which is a good start as these are our targets. We can see that this dataset has two columns, one of which being 'id'. Hopefully we can use this to join our tables later in the EDA process. 

#### The big catch here is seeing the **class imbalance**, we will have to adjust our model accordingly to make up for the lack of values especially within the 'functional needs repair' category. 

### Now we'll review our predictors within the Training_set_values.cvs

In [7]:
# Import training values CSV
df_training_values = pd.read_csv('data/Training_set_values.csv')

In [8]:
df_training_values.shape

(59400, 40)

In [9]:
df_training_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

#### Reviewing the above output, we have a few columns with Null values. Going forward we will review these columns and identify the appropriate way to either replace or drop these columns. Also, we can see that we have an 'id' column which should allow us to join our tables. 

#### Below we will get a clearer understanding of what columns are missing columns. 

In [10]:
df_training_values.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

#### Out of the 39 features 7 of them are missing values. A few items stand out:

* Funder and installer have close to equal amounts of missing values
* subvillage has the least amount of missing values
* scheme_name is missing almost half of the values

#### Since scheme name is missing half of the data we will drop this column as it will not have a significant impact on our model. 

In [11]:
# Dropping column from dataframe
df_training_values.drop('scheme_name', axis = 1, inplace = True)

#### As seen above, 7 columns are missing data. Lets take a deeper look into these columns. 

In [12]:
# Creating a list of columns with missing values
missing_values = ['funder', 'installer', 'subvillage', 'public_meeting',\
                  'scheme_management', 'permit']

# Creating a dataframe with above missing_values
df_training_values[missing_values]

,funder,installer,subvillage,public_meeting,scheme_management,permit
0,Roman,Roman,Mnyusi B,True,VWC,False
1,Grumeti,GRUMETI,Nyamara,NaN,Other,True
2,Lottery Club,World vision,Majengo,True,VWC,True
3,Unicef,UNICEF,Mahakamani,True,VWC,True
4,Action In A,Artisan,Kyanyamisa,True,NaN,True
...,...,...,...,...,...,...
59395,Germany Republi,CES,Kiduruni,True,Water Board,True
59396,Cefa-njombe,Cefa,Igumbilo,True,VWC,True
59397,NaN,NaN,Madungulu,True,VWC,False
59398,Malec,Musa,Mwinyi,True,VWC,True


In [13]:
df_training_values[missing_values].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   funder             55765 non-null  object
 1   installer          55745 non-null  object
 2   subvillage         59029 non-null  object
 3   public_meeting     56066 non-null  object
 4   scheme_management  55523 non-null  object
 5   permit             56344 non-null  object
dtypes: object(6)
memory usage: 2.7+ MB


In [14]:
df_training_values[missing_values].isna().sum()

funder               3635
installer            3655
subvillage            371
public_meeting       3334
scheme_management    3877
permit               3056
dtype: int64

#### We can now see that all of these features are of the dtype object which narrows down our options to dealing with the missing values. What are these features composed of? 

#### To start, lets take a look at our previous mentioned insite of funders and installers having close to the same amount of missing values. 

In [15]:
df_training_values[df_training_values['funder'].isna()]['installer'].isna().value_counts()

True     3582
False      53
Name: installer, dtype: int64

In [16]:
df_training_values[df_training_values['installer'].isna()]['funder'].isna().value_counts()

True     3582
False      73
Name: funder, dtype: int64

#### Looking at the above counts we can compare when one value is null how it effects the other value within the dateframe. 

#### I.E. when 'funder' is null within the dataframe how many of the 'installers' are also labels as null. Above we can see that 3582 are 'installers' are labeled as null while 53 contain and actual value. 

#### With this information we will move forward working with these columns independently of one another.

### Funder:

In [17]:
df_training_values['funder'].value_counts()

Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
                          ... 
Lake Tanganyika Basin        1
Amrefe                       1
Mh.Chiza                     1
Netherla                     1
Lion Clu                     1
Name: funder, Length: 1897, dtype: int64

In [18]:
df_training_values[df_training_values['funder'].isna()]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
34,41583,0.0,2011-02-23,NaN,-41,NaN,39.812912,-7.889986,Msikitini Wa Ijumaa,0,Rufiji,Kilombero B,Pwani,60,63,Mafia,Baleni,100,True,GeoData Consultants Ltd,VWC,False,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
43,19282,0.0,2013-01-15,NaN,1642,NaN,34.967789,-4.628921,Mvae Primary,0,Internal,Mwarufyu,Singida,13,2,Singida Rural,Merya,1,True,GeoData Consultants Ltd,VWC,NaN,1980,mono,mono,motorpump,vwc,user-group,unknown,unknown,unknown,unknown,dry,dry,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
47,13620,0.0,2011-07-27,NaN,0,NaN,33.540607,-9.172905,Mahakamani,0,Lake Nyasa,Mpandapanda,Mbeya,12,4,Rungwe,Kiwira,0,True,GeoData Consultants Ltd,VWC,NaN,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
65,51072,0.0,2013-02-09,NaN,1415,NaN,34.621598,-5.173136,Nyambi,0,Internal,Mfumbu,Singida,13,2,Singida Rural,Naintiri,1,True,GeoData Consultants Ltd,VWC,NaN,1970,mono,mono,motorpump,vwc,user-group,unknown,unknown,unknown,unknown,dry,dry,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
71,17386,0.0,2011-03-31,NaN,0,NaN,34.462228,-8.575780,Kwa Manyusi Mlilo,0,Rufiji,Lembuka,Mbeya,12,7,Mbarali,Mawindi,0,True,GeoData Consultants Ltd,VWC,False,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59357,46563,0.0,2013-02-19,NaN,1635,NaN,34.971841,-5.098362,Shabani,0,Internal,Mampando A,Singida,13,2,Singida Rural,Ntuntu,1,True,GeoData Consultants Ltd,VWC,NaN,1980,nira/tanira,nira/tanira,handpump,vwc,user-group,unknown,unknown,unknown,unknown,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump
59366,55232,0.0,2013-02-02,NaN,1541,NaN,34.765729,-5.027725,Joshoni,0,Internal,Mungumaho,Singida,13,2,Singida Rural,Puma,200,True,GeoData Consultants Ltd,VWC,NaN,2000,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
59370,14796,200.0,2013-01-29,NaN,1154,NaN,30.058731,-4.902633,Village Office,0,Lake Tanganyika,Mazungwe,Kigoma,16,2,Kasulu,Rusesa,1,True,GeoData Consultants Ltd,NaN,False,0,other,other,other,unknown,unknown,pay monthly,monthly,unknown,unknown,unknown,unknown,unknown,other,unknown,other,other
59376,34716,0.0,2013-02-03,NaN,1581,NaN,34.821039,-5.076258,Nasingo,0,Internal,Hu,Singida,13,2,Singida Rural,Dung'unyi,1,True,GeoData Consultants Ltd,VWC,NaN,1990,other,other,other,vwc,user-group,unknown,unknown,unknown,unknown,dry,dry,shallow well,shallow well,groundwater,other,other


#### The 'recorded_by' column contains 'GeoData Consultants Ltd' in each row. Lets take a deeper look at this and see if we are able to drop this row due to a single value. 

### Recorded_by:

In [23]:
df_training_values['recorded_by'].value_counts()

GeoData Consultants Ltd    59400
Name: recorded_by, dtype: int64

#### With only one value for this column it will not have any effect on our model and are able to drop this column.

In [24]:
df_training_values.drop('recorded_by', axis = 1, inplace = True)

### Installer

In [25]:
df_training_values['installer'].value_counts()

DWE                   17402
Government             1825
RWE                    1206
Commu                  1060
DANIDA                 1050
                      ...  
TWIG                      1
Hanja                     1
Kilimarondo Parish        1
AFRICA MUSLIM             1
TWESA/Community           1
Name: installer, Length: 2145, dtype: int64

In [26]:
df_training_values[df_training_values['installer'].isna()]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
34,41583,0.0,2011-02-23,NaN,-41,NaN,39.812912,-7.889986,Msikitini Wa Ijumaa,0,Rufiji,Kilombero B,Pwani,60,63,Mafia,Baleni,100,True,VWC,False,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,salty,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
43,19282,0.0,2013-01-15,NaN,1642,NaN,34.967789,-4.628921,Mvae Primary,0,Internal,Mwarufyu,Singida,13,2,Singida Rural,Merya,1,True,VWC,NaN,1980,mono,mono,motorpump,vwc,user-group,unknown,unknown,unknown,unknown,dry,dry,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
47,13620,0.0,2011-07-27,NaN,0,NaN,33.540607,-9.172905,Mahakamani,0,Lake Nyasa,Mpandapanda,Mbeya,12,4,Rungwe,Kiwira,0,True,VWC,NaN,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
65,51072,0.0,2013-02-09,NaN,1415,NaN,34.621598,-5.173136,Nyambi,0,Internal,Mfumbu,Singida,13,2,Singida Rural,Naintiri,1,True,VWC,NaN,1970,mono,mono,motorpump,vwc,user-group,unknown,unknown,unknown,unknown,dry,dry,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
71,17386,0.0,2011-03-31,NaN,0,NaN,34.462228,-8.575780,Kwa Manyusi Mlilo,0,Rufiji,Lembuka,Mbeya,12,7,Mbarali,Mawindi,0,True,VWC,False,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59357,46563,0.0,2013-02-19,NaN,1635,NaN,34.971841,-5.098362,Shabani,0,Internal,Mampando A,Singida,13,2,Singida Rural,Ntuntu,1,True,VWC,NaN,1980,nira/tanira,nira/tanira,handpump,vwc,user-group,unknown,unknown,unknown,unknown,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump
59366,55232,0.0,2013-02-02,NaN,1541,NaN,34.765729,-5.027725,Joshoni,0,Internal,Mungumaho,Singida,13,2,Singida Rural,Puma,200,True,VWC,NaN,2000,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
59370,14796,200.0,2013-01-29,NaN,1154,NaN,30.058731,-4.902633,Village Office,0,Lake Tanganyika,Mazungwe,Kigoma,16,2,Kasulu,Rusesa,1,True,NaN,False,0,other,other,other,unknown,unknown,pay monthly,monthly,unknown,unknown,unknown,unknown,unknown,other,unknown,other,other
59376,34716,0.0,2013-02-03,NaN,1581,NaN,34.821039,-5.076258,Nasingo,0,Internal,Hu,Singida,13,2,Singida Rural,Dung'unyi,1,True,VWC,NaN,1990,other,other,other,vwc,user-group,unknown,unknown,unknown,unknown,dry,dry,shallow well,shallow well,groundwater,other,other


#### Again, reviewing the above dataframe, nothing jumps out of importance. We will continue on and conclude our null value adjustments at a later time.

### Subvillage

In [29]:
df_training_values['subvillage'].value_counts()

Madukani      508
Shuleni       506
Majengo       502
Kati          373
Mtakuja       262
             ... 
Kiimawe         1
Kafwandi        1
Hindawashi      1
Kwa Tupa        1
Mwamasha        1
Name: subvillage, Length: 19287, dtype: int64

In [30]:
df_training_values[df_training_values['subvillage'].isna()]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
342,21127,0.0,2011-03-16,Government Of Tanzania,0,North,36.557631,-6.233394,Kwa Mihinzo,0,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Sagara,0,True,VWC,False,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,dry,dry,spring,spring,groundwater,communal standpipe multiple,communal standpipe
360,51558,0.0,2011-03-25,Commu,0,Commu,36.416701,-6.220157,Kwa Emanuel,0,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Kongwa Urban,0,True,VWC,False,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,dry,dry,spring,spring,groundwater,communal standpipe,communal standpipe
379,53847,0.0,2011-03-20,World Bank,0,Rhobi,36.729383,-6.084255,Kwa Dimanyi,0,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Pandambili,0,True,VWC,False,0,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
565,27334,0.0,2011-03-18,World Bank,0,Rhoda,36.696881,-5.993192,Mpande,0,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Njoge,0,True,VWC,False,0,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
966,17088,0.0,2011-03-11,Water,0,Commu,36.322623,-6.030500,Kwa Charles,0,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Sejeli,0,True,VWC,True,0,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket,salty,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59008,16353,0.0,2011-03-23,Commu,0,Commu,36.770490,-6.288555,Kwa Mlima,0,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Mlali,0,True,VWC,False,0,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
59091,45206,0.0,2011-03-14,Lvia,0,Commu,36.407974,-5.715084,Kwa Emson,0,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Zoissa,0,True,VWC,False,0,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket,salty,salty,dry,dry,machine dbh,borehole,groundwater,communal standpipe,communal standpipe
59105,12248,0.0,2011-03-19,World Bank,0,Rhobi,36.889359,-5.959966,Kwa Mahimbo,0,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Pandambili,0,True,VWC,False,0,submersible,submersible,submersible,vwc,user-group,never pay,never pay,salty,salty,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
59215,46441,0.0,2011-03-19,World Bank,0,Rhobi,36.854216,-6.010508,Kwa Sila,0,Wami / Ruvu,NaN,Dodoma,1,3,Kongwa,Pandambili,0,True,VWC,False,0,submersible,submersible,submersible,vwc,user-group,never pay,never pay,salty,salty,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe


#### One observation I found was that the 'region' contains 'Dodoma' multiple times. Lets look at the value counts for this.  

In [31]:
df_training_values[df_training_values['subvillage'].isna()]['region'].value_counts()

Dodoma    361
Mwanza     10
Name: region, dtype: int64

#### With Dodoma being the majority value here we will keep this in mind when replacing the values later on. 

### Public Meeting:

In [34]:
df_training_values['public_meeting'].value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

In [33]:
df_training_values[df_training_values['public_meeting'].isna()]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466e+00,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,Other,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
18,34169,0.0,2011-07-22,Hesawa,1162,DWE,32.920154,-1.947868e+00,Ngomee,0,Lake Victoria,Center,Mwanza,19,1,Ukerewe,Ilangala,1000,NaN,NaN,True,1999,other,other,other,vwc,user-group,never pay,never pay,milky,milky,insufficient,insufficient,spring,spring,groundwater,other,other
21,6091,0.0,2013-02-10,Dwsp,0,DWE,0.000000,-2.000000e-08,Muungano,0,Lake Victoria,Ibabachegu,Shinyanga,17,1,Bariadi,Ikungulyabashashi,0,NaN,WUG,False,0,swn 80,swn 80,handpump,wug,user-group,unknown,unknown,unknown,unknown,unknown,unknown,shallow well,shallow well,groundwater,hand pump,hand pump
58,24593,0.0,2013-01-22,Kkkt,1703,KKKT,35.561346,-3.806879e+00,Kwa Iyora Mgirigisi,0,Internal,Ants B,Manyara,21,3,Mbulu,Bargish,456,NaN,WUG,True,2000,nira/tanira,nira/tanira,handpump,wug,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
73,35715,0.0,2013-02-27,Hesawa,1565,HESAWA,34.657034,-1.888020e+00,Machumbe,0,Lake Victoria,Mtakuja,Mara,20,2,Serengeti,Manchira,200,NaN,Other,True,1991,nira/tanira,nira/tanira,handpump,wug,user-group,other,other,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59306,71419,0.0,2013-01-16,Cocern,1296,TWESA,30.930385,-3.300592e+00,Kwa Mhunda,0,Lake Tanganyika,Nguvu Kazi A,Kigoma,16,1,Kibondo,Kakonko,1,NaN,NaN,False,2013,gravity,gravity,gravity,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,spring,spring,groundwater,improved spring,improved spring
59310,32815,0.0,2013-04-04,Tasaf,892,TASAF,37.744090,-4.062296e+00,Ward Office,0,Pangani,Same,Kilimanjaro,3,3,Same,Same Urban,65,NaN,NaN,NaN,0,other,other,other,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,other,unknown,other,other
59324,748,0.0,2013-01-22,World Vision,0,World Vision,0.000000,-2.000000e-08,Mwazwilo,0,Lake Victoria,Mbita,Shinyanga,17,1,Bariadi,Mbita,0,NaN,WUG,False,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
59344,6450,0.0,2013-10-03,NaN,1303,NaN,36.900911,-3.111477e+00,Pentecoste Swidish,0,Internal,Madukani,Arusha,2,7,Meru,Ngarenanyuki,230,NaN,NaN,NaN,0,gravity,gravity,gravity,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,spring,spring,groundwater,communal standpipe,communal standpipe


#### Again, reviewing the above dataframe, nothing jumps out of importance. We will continue on and conclude our null value adjustments at a later time.

In [35]:
df_training_values['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [40]:
df_training_values[df_training_values['scheme_management'].isna()]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
7,54551,0.0,2012-10-09,Rwssp,0,DWE,32.620617,-4.226198,Tushirikiane,0,Lake Tanganyika,Nyawishi Center,Shinyanga,17,3,Kahama,Chambo,0,True,NaN,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
9,46144,0.0,2011-08-03,Isingiro Ho,0,Artisan,30.626991,-1.257051,Kwapeto,0,Lake Victoria,Mkonomre,Kagera,18,1,Karagwe,Kaisho,0,True,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
18,34169,0.0,2011-07-22,Hesawa,1162,DWE,32.920154,-1.947868,Ngomee,0,Lake Victoria,Center,Mwanza,19,1,Ukerewe,Ilangala,1000,NaN,NaN,True,1999,other,other,other,vwc,user-group,never pay,never pay,milky,milky,insufficient,insufficient,spring,spring,groundwater,other,other
27,20145,0.0,2011-07-11,Hesawa,0,DWE,31.104257,-1.735361,Kwawilliam,0,Lake Victoria,Chakahaya,Kagera,18,1,Karagwe,Nyaishozi,0,True,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59329,27957,0.0,2013-02-09,Rwssp,0,DWE,32.597166,-3.812227,Kasela,0,Lake Victoria,Kasela,Shinyanga,17,3,Kahama,Malunga,0,True,NaN,True,0,other,other,other,wug,user-group,unknown,unknown,milky,milky,insufficient,insufficient,shallow well,shallow well,groundwater,other,other
59344,6450,0.0,2013-10-03,NaN,1303,NaN,36.900911,-3.111477,Pentecoste Swidish,0,Internal,Madukani,Arusha,2,7,Meru,Ngarenanyuki,230,NaN,NaN,NaN,0,gravity,gravity,gravity,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,spring,spring,groundwater,communal standpipe,communal standpipe
59362,18917,0.0,2011-03-02,Samlo,2190,SHIPO,34.716812,-9.537797,none,0,Lake Nyasa,Mnogelwa,Iringa,11,4,Njombe,Luponde,60,True,NaN,False,2007,other - rope pump,rope pump,rope pump,private operator,commercial,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
59370,14796,200.0,2013-01-29,NaN,1154,NaN,30.058731,-4.902633,Village Office,0,Lake Tanganyika,Mazungwe,Kigoma,16,2,Kasulu,Rusesa,1,True,NaN,False,0,other,other,other,unknown,unknown,pay monthly,monthly,unknown,unknown,unknown,unknown,unknown,other,unknown,other,other


In [ ]:
df_training_values['permit'].value_counts()

## Data Preparation

### For each column we will create two variables for modeling. One with the mode value for each column and one with a newly created variable denoted 'other'.

In [ ]:
# Creating new dataframe
df_training_val_mode = df_training_values.copy()
df_training_val_other = df_training_values.copy()


In [ ]:
# Filling NAN values to 'Other'

df_training_val_other['funder'] = df_training_val_other['funder']\
                            .replace(np.nan, 'Other', regex = True)

df_training_val_other['installer'] = df_training_val_other['installer']\
                                .replace(np.nan, 'Other', regex = True)

df_training_val_other['subvillage'] = df_training_val_other['subvillage']\
                                    .replace(np.nan, 'Other', regex = True)

df_training_val_other['public_meeting'] = df_training_val_other['public_meeting']\
                                            .replace(np.nan, 'Other', regex = True)

df_training_val_other['scheme_management'] = df_training_val_other['scheme_management']\
                                                .replace(np.nan, 'Other', regex = True)

df_training_val_other['permit'] = df_training_val_other['permit']\
                            .replace(np.nan, 'Other', regex = True)


In [ ]:
# Filling NAN values with most common feature based on count

df_training_val_mode['funder'].fillna(df_training_val_mode['funder']\
                        .value_counts().index[0], inplace = True)

df_training_val_mode['installer'].fillna(df_training_val_mode['installer']\
                                .value_counts().index[0], inplace = True)

df_training_val_mode['subvillage'].fillna(df_training_val_mode['subvillage']\
                                    .value_counts().index[0], inplace = True)

df_training_val_mode['public_meeting'].fillna(df_training_val_mode['public_meeting']\
                                            .value_counts().index[0], inplace = True)

df_training_val_mode['scheme_management'].fillna(df_training_val_mode['scheme_management']\
                                                 .value_counts().index[0], inplace = True)

df_training_val_mode['permit'].fillna(df_training_val_mode['permit']\
                            .value_counts().index[0], inplace = True)

In [ ]:
df_training_val_mode.isna().sum()

In [ ]:
df_training_val_other.isna().sum()

### Joining Tables

#### Now lets merge the tables so we only have two data sets to work with. To start, both dataframes have an ID column so we will create a new column on our target set and drop the identical column.

In [ ]:
df_training_labels['id_2'] = df_training_labels['id']
df_training_labels.drop('id', axis = 1, inplace = True)

#### Next we will join our tables and create two dataframes for mode and other

In [ ]:
df_mode = pd.concat([df_training_val_mode, df_training_labels], join = 'inner', axis = 1)
df_other = pd.concat([df_training_val_other, df_training_labels], join = 'inner', axis = 1)

In [ ]:
df_mode.shape

In [ ]:
df_mode[df_mode['id'] == df_mode['id_2']]

In [ ]:
df_other[df_other['id'] == df_other['id_2']]

#### As seen above our total rows equal that of the normal dataframe so we can conclude that our merges have been successful and we can drop our id_2 column.

In [ ]:
df_mode.drop(['id_2'], axis = 1, inplace = True)
df_other.drop(['id_2'], axis = 1, inplace = True)

### Additional Columns to Drop

#### The Id columns and date_recorded are considered admin columns and will not have much predictive power in our model therefore we can drop these columns. 

In [ ]:
df_mode.drop(['id', 'date_recorded'], axis = 1, inplace = True)
df_other.drop(['id', 'date_recorded'], axis = 1, inplace = True)

In [ ]:
def get_totals(dataframe, filter_column, filter_groupby):

        '''
        **** filter_column & filter_groupby need to be passed
        as strings ****

        1. get_totals will calculate the sum of the variables
        within a column and return a new column with the 
        sum of their total occurances in the dataframe
        
        2. get_totals will calulate the percentage of the 
        values column vs the total values

        dataframe = pandas dataframe
        filter_column = column to filter by
        filter_groupby = groupby column to filter by

        '''

        df_new = pd.DataFrame(dataframe.groupby(filter_groupby)[filter_column].value_counts())
        df_new[f'{filter_groupby}_values'] = df_new[filter_column]
        df_new.drop(filter_column, axis = 1, inplace = True)
        df_new.reset_index(inplace = True)

        types = set()

        for idx, value in enumerate(df_new[f'{filter_groupby}_values']):
            for type_ in df_new[filter_column]:
                types.add(type_)
            
        total_values = {}
            
        for value in types:
            total_values[value] = df_new[df_new[filter_column] == value][f'{filter_groupby}_values'].sum()

        df_new[f'{filter_groupby}_total_values'] = df_new[filter_column].map(total_values)

        df_new[f'{filter_groupby}_percentage'] = df_new[f'{filter_groupby}_values'] / df_new[f'{filter_groupby}_total_values']
            
        return df_new


In [ ]:
# function_df = df.drop('status_group', axis = 1)

# percentage_dict = {}

# for idx, column in enumerate(function_df.columns):
#     percentage_dict[column] = get_totals(df, column, 'status_group')

# pickle_out = open('percentage_dict.pickle', 'wb')
# pickle.dump(percentage_dict, pickle_out)

In [ ]:
pickle_in = open('percentage_dict.pickle', 'rb')

percentage_dict = pickle.load(pickle_in)

In [ ]:
df_mode.columns.tolist()

In [ ]:
percentage_dict['region']

In [ ]:
sns.set_theme(style='darkgrid')

extraction_type = percentage_dict['region']
extraction_type_funct = extraction_type[extraction_type['status_group'] == 'non functional']
extraction_type_funct = extraction_type_funct.sort_values('status_group_percentage', ascending = False)

fig, ax = plt.subplots(figsize=(10,10))
ax.barh(extraction_type_funct['region'], extraction_type_funct['status_group_percentage'])
plt.xlabel('Percentage of Non Functional Wells')
plt.ylabel('Region')
plt.yticks(rotation=30)
plt.title('Percentage of Non Functional Wells by Region')
plt.savefig('saved_objects/non_funct_region');

In [ ]:
sns.set_theme(style='darkgrid')

extraction_type = percentage_dict['basin']
extraction_type_funct = extraction_type[extraction_type['status_group'] == 'functional']
extraction_type_funct = extraction_type_funct.sort_values('status_group_percentage', ascending = False)

fig, ax = plt.subplots(figsize=(10,10))
ax.barh(extraction_type_funct['basin'], extraction_type_funct['status_group_percentage'])
plt.title('Percentage of Non Functional Wells by Water Quality')
plt.savefig('saved_objects/non_water_quality');

In [ ]:
percentage_dict['water_quality']

In [ ]:
sns.set_theme(style='darkgrid')

extraction_type = percentage_dict['water_quality']
extraction_type_funct = extraction_type[extraction_type['status_group'] == 'functional']
extraction_type_funct = extraction_type_funct.sort_values('status_group_percentage', ascending = False)

fig, ax = plt.subplots(figsize=(10,10))
ax.barh(extraction_type_funct['water_quality'], extraction_type_funct['status_group_percentage'])
plt.yticks(rotation = 45)
plt.title('Percentage of Non Functional Wells by Water Quality')
plt.savefig('saved_objects/non_water_quality');

In [ ]:
extraction_type = percentage_dict['extraction_type']
extraction_type_funct = extraction_type[extraction_type['status_group'] == 'functional']
extraction_type_funct = extraction_type_funct.sort_values('status_group_percentage', ascending = False)

fig, ax = plt.subplots(figsize=(10,10))
ax.barh(extraction_type_funct['extraction_type'], extraction_type_funct['status_group_percentage'])

In [ ]:
source = percentage_dict['source']
functional = source[source['status_group'] == 'functional']
functional

plt.bar(source['source'], source['status_group_percentage'])
plt.xticks(rotation=45, ha='right')
plt.title('Top Functional Wells by Source')
plt.savefig('saved_objects/source_bar');

In [ ]:
source = percentage_dict['payment']
functional = source[source['status_group'] == 'functional']
functional

plt.bar(source['payment'], source['status_group_percentage'])
plt.xticks(rotation=45, ha='right')
plt.title('Money?')
plt.savefig('saved_objects/money_bar');

### First Simple Model

#### To start our modeling process we will use only our integers and floats.

In [ ]:
X_mode_fsm = df_mode.select_dtypes(['int64', 'float64'])
y_mode_fsm = df_mode['status_group']

X_other_fsm = df_other.select_dtypes(['int64','float64'])
y_other_fsm = df_other['status_group']

In [ ]:
X_mode_train, X_mode_test, y_mode_train, y__mode_test = train_test_split(X_mode_fsm,y_mode_fsm, random_state = 42, stratify = y_mode_fsm)

X_other_train, X_other_test, y_other_train, y_other_test = train_test_split(X_other_fsm,y_other_fsm, random_state = 42, stratify = y_mode_fsm)


dtc_mode = DecisionTreeClassifier()
dtc_other = DecisionTreeClassifier()

dtc_mode.fit(X_mode_train, y_mode_train)
dtc_other.fit(X_other_train, y_other_train)

In [ ]:
print(dtc_mode.score(X_mode_train, y_mode_train))
print(dtc_other.score(X_other_train, y_other_train))

In [ ]:
y_hat_mode = dtc_mode.predict(X_mode_train)
y_hat_other = dtc_other.predict(X_other_train)

In [ ]:
print('mode recall:', recall_score(y_mode_train, y_hat_mode, average = 'macro'))
print('mode precision:', precision_score(y_mode_train, y_hat_mode, average = 'macro'))
print('mode f1 score:', f1_score(y_mode_train, y_hat_mode, average = 'macro'))

print('---------------------------------------------------------')

print('ohter recall:', recall_score(y_other_train, y_hat_other, average = 'macro'))
print('other precision:', precision_score(y_other_train, y_hat_other, average = 'macro'))
print('other f1 score:', f1_score(y_other_train, y_hat_other, average = 'macro'))

In [ ]:
cross_val_score(dtc_mode, X_mode_train, y_mode_train, cv = 3, scoring = 'recall_macro')

In [ ]:
cross_val_score(dtc_other, X_mode_train, y_mode_train, cv = 3, scoring = 'recall_macro')

### The cross val scores are pretty consitent across the folds. This doesnt give us much insight as far as our NAN replacements in during the EDA. 

### Model Exploration

#### Now that we have our baseline established we will loop through other models to see if we can get better results between our two dataframes.

### 'Mode DF Loop

In [ ]:
# model_selection = [LogisticRegression(random_state = 42, max_iter = 1000, n_jobs = -1),\
#                    RandomForestClassifier(random_state = 42, n_jobs = -1),\
#                    DecisionTreeClassifier(), KNeighborsClassifier(n_jobs = -1), 
#                   SVC(random_state = 42)]

# vanilla_models = {}

# for idx_mode, model in enumerate(model_selection):
#     vanilla_models[idx_mode] = model.fit(X_mode_train, y_mode_train)

In [ ]:
# with open('baseline_models_dict.pickle', 'wb') as f:
#     pickle.dump(vanilla_models, f)

In [ ]:
# scores = {}
# recall_macro = {}

# for key, val in enumerate(vanilla_models.values()):
#     scores[val] = val.score(X_mode_train, y_mode_train)
#     recall_macro[val] = np.mean(cross_val_score(val, X_mode_train, y_mode_train, scoring = 'recall_macro'))

In [ ]:
# with open('baseline_scores_mode_dict.pickle', 'wb') as f:
#     pickle.dump(scores, f)

In [ ]:
# with open('baseline_recall_macro_mode_dict.pickle', 'wb') as f:
#     pickle.dump(recall_macro, f)

### 'Other' DF Loop

In [ ]:
# model_selection = [LogisticRegression(random_state = 42, max_iter = 1000, n_jobs = -1),\
#                    RandomForestClassifier(random_state = 42, n_jobs = -1),\
#                    DecisionTreeClassifier(), KNeighborsClassifier(n_jobs = -1), 
#                   SVC(random_state = 42)]

# vanilla_models = {}

# for idx_other, model in enumerate(model_selection):
#     vanilla_models[idx_other] = model.fit(X_other_train, y_other_train)

In [ ]:
# scores_other = {}
# recall_macro_other = {}

# for key, val in enumerate(vanilla_models.values()):
#     scores_other[val] = val.score(X_other_train, y_other_train)
#     recall_macro_other[val] = np.mean(cross_val_score(val, X_other_train, y_other_train, scoring = 'recall_macro'))

In [ ]:
# with open('baseline_recall_macro_other_dict.pickle', 'wb') as f:
#     pickle.dump(recall_macro_other, f)

In [ ]:
# with open('baseline_scores_macro_other_dict.pickle', 'wb') as f:
#     pickle.dump(scores_other, f)

### Vanilla Models

In [ ]:
with open('baseline_models_dict.pickle', 'rb') as f:
    baseline_models = pickle.load(f)
baseline_models

### 'Mode' Results

In [ ]:
with open('baseline_scores_mode_dict.pickle', 'rb') as f:
    baseline_mode_scores = pickle.load(f)
baseline_mode_scores

In [ ]:
with open('baseline_recall_macro_mode_dict.pickle', 'rb') as f:
    baseline_mode_recall = pickle.load(f)
    
baseline_mode_recall

### 'Other' Results

In [ ]:
with open('baseline_scores_macro_other_dict.pickle', 'rb') as f:
    baseline_other_scores = pickle.load(f)
baseline_other_scores

In [ ]:
with open('baseline_recall_macro_other_dict.pickle', 'rb') as f:
    baseline_other_recall = pickle.load(f)
baseline_other_recall

#### Based on the scores above, our scores are the best using RandomForestClassifier. This will be the model we use moving forward. There isn't any difference between our two dataframes so either one will work. We will move forward using the 'Mode' dataframe. 

#### For readability we will get all of our baseline metrics in one place.

In [ ]:
# Review scores for both models
rfc_mode_yhat = baseline_models[1].predict(X_mode_train)

# All baseline metrics
print('RFC Mode Recall:', recall_score(y_mode_train, rfc_mode_yhat, average = 'macro'))
print('RFC Mode Precision:', precision_score(y_mode_train, rfc_mode_yhat, average = 'macro'))
print('RFC Mode F1 score:', f1_score(y_mode_train, rfc_mode_yhat, average = 'macro'))
print('Baseline Cross_Val_Score:', cross_val_score(baseline_models[1], X_mode_train, y_mode_train,\
                                                   cv=5, n_jobs=-1, scoring = 'recall_macro'))


### Our Recall, Precision, and F1 score all look good but our true test is the cross validation score. We can see a significant drop in our predicting capabilities. Hopefully we wil lbe able to improve on this going forward.


### Also, since our stakeholder is concerned with pump failures we need to avoid False Negatives. I.E. we do not want to say the well is broken when it in fact it is operational. Therefore we need to focus on our recall score and tune our model appropriately which is why we've used the recall_macro score so far. 

### Our models above only utilized our numerical values. We will now begin using our categorical features and identify feature importance. 

In [ ]:
# Model #1

# Separate data by target and predictors
X_cat = df_mode.drop('status_group', axis = 1)
y_cat = df_mode['status_group']

# Perform train test split
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y_cat, random_state = 42, stratify = y_cat)

# One hot encoded categorical data
ohe = OneHotEncoder(drop = 'first')

# Select initial parameters
df_feat_import = X_train_cat[['extraction_type', 'management', 'payment', 'water_quality', 'source', 'source_class', 'region_code', 'district_code']]

# fit transform data
X_mode_train_enc = ohe.fit_transform(df_feat_import)

# Instantiate model
rfc_feat_import = RandomForestClassifier(random_state = 42, class_weight= 'balanced', n_jobs = -1)


# Fit encoded data to model
rfc_feat_import.fit(X_mode_train_enc, y_train_cat)

# Model score
rfc_feat_import.score(X_mode_train_enc, y_train_cat)

In [ ]:
# Predict on training data
rfc_yhat_1 = rfc_feat_import.predict(X_mode_train_enc)

# Recall score on training data
recall_score(y_train_cat, rfc_yhat_1, average='macro')

In [ ]:
# Precision score on training data
precision_score(y_train_cat, rfc_yhat_1, average='macro')

In [ ]:
#F1 Score on training data
f1_score(y_train_cat, rfc_yhat_1, average='macro')

In [ ]:
# 5-fold cross validation
cross_val_score(rfc_feat_import, X_mode_train_enc, y_train_cat, cv = 5, scoring = 'recall_macro')

In [ ]:
# Confusion matrix
plot_confusion_matrix(rfc_feat_import, X_mode_train_enc, y_train_cat)
plt.grid(None);

In [ ]:
# ROC AUC via Yellowbrick
visualizer = ROCAUC(rfc_feat_import)
visualizer.fit(X_mode_train_enc, y_train_cat)
visualizer.score(X_mode_train_enc, y_train_cat)
visualizer.show();

#### Adding features to see if our model improves all other steps are a repeat from above

In [ ]:
# Model #2

ohe = OneHotEncoder(drop = 'first')

df_feat_import_2 = X_train_cat[[
 'source_type',
 'region',
 'district_code',
 'public_meeting',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'payment_type',
 'quantity_group',
 'source',
 'source_class',
 'waterpoint_type_group']]

X_mode_train_enc_2 = ohe.fit_transform(df_feat_import_2)

rfc_feat_import_2 = RandomForestClassifier(random_state = 42, class_weight= 'balanced', n_jobs = -1)

rfc_feat_import_2.fit(X_mode_train_enc_2, y_train_cat)

rfc_feat_import_2.score(X_mode_train_enc_2, y_train_cat)

In [ ]:
rfc_yhat_2 = rfc_feat_import_2.predict(X_mode_train_enc_2)

recall_score(y_train_cat, rfc_yhat_2, average = 'macro')

In [ ]:
# Precision score on training data
precision_score(y_train_cat, rfc_yhat_2, average='macro')

In [ ]:
#F1 Score on training data
f1_score(y_train_cat, rfc_yhat_2, average='macro')

In [ ]:
cross_val_score(rfc_feat_import_2, X_mode_train_enc_2, y_train_cat, cv = 5, scoring = 'recall_macro')

In [ ]:
plot_confusion_matrix(rfc_feat_import_2, X_mode_train_enc_2, y_train_cat)
plt.grid(None);

In [ ]:
visualizer = ROCAUC(rfc_feat_import_2)
visualizer.fit(X_mode_train_enc_2, y_train_cat)
visualizer.score(X_mode_train_enc_2, y_train_cat)
visualizer.show();

#### Adding features to see if our model improves all other steps are a repeat from above

In [ ]:
# Model #3

ohe = OneHotEncoder(drop = 'first')

df_feat_import_3 = X_train_cat[['region_code', 'source_type', 'basin', 'region', 'region_code', 'district_code',\
                   'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type',\
                  'extraction_type_group', 'extraction_type_class','management',\
                   'management_group', 'payment', 'payment_type', 'water_quality',\
                  'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
                   'source_class', 'waterpoint_type', 'waterpoint_type_group']]

X_mode_train_enc_3 = ohe.fit_transform(df_feat_import_3)

rfc_feat_import_3 = RandomForestClassifier(random_state = 42, class_weight = 'balanced',n_jobs = -1)

rfc_feat_import_3.fit(X_mode_train_enc_3, y_train_cat)

rfc_feat_import_3.score(X_mode_train_enc_3, y_train_cat)

In [ ]:
rfc_yhat_3 = rfc_feat_import_3.predict(X_mode_train_enc_3)

recall_score(y_train_cat, rfc_yhat_3, average = 'macro')

In [ ]:
precision_score(y_train_cat, rfc_yhat_3, average='macro')

In [ ]:
f1_score(y_train_cat, rfc_yhat_3, average='macro')

In [ ]:
cross_val_score(rfc_feat_import_3, X_mode_train_enc_3, y_train_cat, cv = 5, scoring = 'recall_macro')

In [ ]:
plot_confusion_matrix(rfc_feat_import_3, X_mode_train_enc_3, y_train_cat)
plt.grid(None);

In [ ]:
visualizer = ROCAUC(rfc_feat_import_3)
visualizer.fit(X_mode_train_enc_3, y_train_cat)
visualizer.score(X_mode_train_enc_3, y_train_cat)
visualizer.show();

In [ ]:
# param_grid = {
#  'max_depth': [3,10, None],
#  'criterion': ['gini', 'entropy'],
#  'min_samples_leaf': [1, 2, 4],
#  'n_estimators': [100, 500],
#  'class_weight': ['balanced', 'balanced_subsample'],
#  'n_jobs': [-1]
# }

# grid_search_cat = GridSearchCV(rfc_feat_import_3, param_grid, n_jobs=-1, cv = 3, return_train_score=True)

# grid_search_cat.fit(X_mode_train_enc_3, y_train)

In [ ]:
# with open('grid_search_cat_all', 'wb') as f:
#     pickle.dump(grid_search_cat, f)

In [ ]:
with open('grid_search_cat_all', 'rb') as f:
    grid_search_cat = pickle.load(f)

In [ ]:
grid_search_cat.best_params_

In [ ]:
final_cat_model = RandomForestClassifier(criterion='entropy', max_depth=None, min_samples_leaf=1,\
                                         n_estimators=500,random_state = 42, class_weight = 'balanced',n_jobs = -1)

final_cat_model.fit(X_mode_train_enc_3, y_train_cat)

final_cat_model.score(X_mode_train_enc_3, y_train_cat)

In [ ]:
final_cat_model_yhat = final_cat_model.predict(X_mode_train_enc_3)

recall_score(y_train_cat,final_cat_model_yhat, average = 'macro')

In [ ]:
precision_score(y_train_cat,final_cat_model_yhat, average='macro')

In [ ]:
f1_score(y_train_cat, final_cat_model_yhat, average='macro')

In [ ]:
cross_val_score(final_cat_model, X_mode_train_enc_3, y_train_cat, cv = 5, scoring = 'recall_macro')

In [ ]:
plot_confusion_matrix(final_cat_model, X_mode_train_enc_3, y_train_cat)
plt.grid(None);

In [ ]:
visualizer = ROCAUC(final_cat_model)
visualizer.fit(X_mode_train_enc_3, y_train_cat)
visualizer.score(X_mode_train_enc_3, y_train_cat)
visualizer.show();

### Interesting results between our first three models with categorical features. Since we're looking for recall, model 1 and model 2 are our best performers with model 3 showing the lowest recall. Also, we can see that the cross validation score dropped by 10%. This model is definitely going in the trash. 

### Since we're one hot encoding all of these features we will start with model 1 and provide the model with less features. 

### Categorical & Numerical Data

### Finally we will build a pipeline to incorporate our numberical data.

In [ ]:
df_mode

In [ ]:
X = df_mode[['region_code', 'source_type', 'basin', 'region', 'district_code',\
                 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type',\
                 'extraction_type_group', 'extraction_type_class','management',\
                 'management_group', 'payment', 'payment_type', 'water_quality',\
                 'quantity', 'quantity_group', 'source',\
                 'source_class', 'waterpoint_type', 'waterpoint_type_group',\
                 'longitude', 'latitude']]

y = df_mode['status_group']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42, stratify = y)

In [ ]:
numeric_features = ['population', 'construction_year', 'gps_height', 'longitude', 'latitude']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['region_code', 'source_type', 'basin', 'region', 'district_code',\
                 'public_meeting', 'scheme_management', 'permit', 'extraction_type',\
                 'extraction_type_group', 'extraction_type_class','management',\
                 'management_group', 'payment', 'payment_type', 'water_quality',\
                 'quantity', 'quantity_group', 'source',\
                 'source_class', 'waterpoint_type', 'waterpoint_type_group']

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(random_state = 42, class_weight= 'balanced', n_jobs = -1))])


clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
# with open('clf_model', 'wb') as f:
#     pickle.dump(clf, f)

In [ ]:
with open('clf_model', 'rb') as f:
    clf = pickle.load(f)

In [ ]:
y_hat_test = clf.predict(X_test)

recall_scoore(y_train, y_hat_test, average ='macro')

In [ ]:
precision_score(y_train, y_hat, average = 'macro')

In [ ]:
f1_score(y_train, y_hat, average = 'macro')

In [ ]:
cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1, scoring = 'recall_macro')

In [ ]:
plot_confusion_matrix(clf, X_train, y_train)
plt.grid(None);

In [ ]:
visualizer = ROCAUC(clf)
visualizer.fit(X_train, y_train)
visualizer.score(X_train, y_train)
visualizer.show();

In [ ]:
param_grid = {
 'classifier__max_depth': [3,10, None],
 'classifier__criterion': ['gini', 'entropy'],
 'classifier__min_samples_leaf': [1, 2, 4],
 'classifier__n_estimators': [100, 500],
 'classifier__class_weight': ['balanced', 'balanced_subsample'],
 'classifier__n_jobs': [-1]
}

grid_search = GridSearchCV(clf, param_grid, n_jobs=-1, cv = 3, return_train_score=True)

grid_search.fit(X_train, y_train)

In [ ]:
# with open('grid_search_all', 'wb') as f:
#     pickle.dump(grid_search, f)

In [ ]:
with open('grid_search_all', 'rb') as f:
    grid_search = pickle.load(f)

In [ ]:
grid_search.best_params_

In [ ]:
numeric_features = ['population', 'construction_year', 'gps_height', 'longitude', 'latitude']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['region_code', 'source_type', 'basin', 'region', 'district_code',\
                 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type',\
                 'extraction_type_group', 'extraction_type_class','management',\
                 'management_group', 'payment', 'payment_type', 'water_quality',\
                 'quantity', 'quantity_group', 'source', 'source_type',\
                 'source_class', 'waterpoint_type', 'waterpoint_type_group']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
final_model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(criterion='entropy', max_depth=None, min_samples_leaf=1,\
                                                            n_estimators=1,random_state = 42, class_weight= 'balanced', n_jobs = -1))])


final_model.fit(X_train, y_train)
final_model.score(X_test, y_test)

In [ ]:
final_yhat = final_model.predict(X_train)
recall_score(y_train, final_yhat, average = 'macro')

In [ ]:
precision_score(y_train, final_yhat, average = 'macro')

In [ ]:
f1_score(y_train, final_yhat, average = 'macro')

In [ ]:
cross_val_score(final_model, X_train, y_train, cv=5, n_jobs=-1, scoring = 'recall_macro')


In [ ]:
plot_confusion_matrix(grid_search, X_train, y_train)
plt.grid(None);

In [ ]:
visualizer = ROCAUC(grid_search)
visualizer.fit(X_train, y_train)
visualizer.score(X_train, y_train)
visualizer.show();

## Testing Scores

In [ ]:
df_feat_import_2_test = X_test_cat[[
 'source_type',
 'region',
 'district_code',
 'public_meeting',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'payment_type',
 'quantity_group',
 'source',
 'source_class',
 'waterpoint_type_group']]

ohe_2_test = OneHotEncoder(drop='first')
ohe_X_test_2_enc = ohe_2_test.fit_transform(df_feat_import_2_test)

model_2_yhat = rfc_feat_import_2.predict(ohe_X_test_2_enc)

print(cross_val_score(rfc_feat_import_2, ohe_X_test_2_enc, y_test_cat, scoring = 'recall_macro', n_jobs=-1, cv = 5))
print(recall_score(y_test_cat, model_2_yhat, average = 'macro'))

In [ ]:
np.mean(
[0.65105011, 0.67144316, 0.63116511, 0.66113869, 0.64810911])

In [ ]:
X = df_mode[[
 'source_type',
 'region',
 'district_code',
 'public_meeting',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'payment_type',
 'quantity_group',
 'source',
 'source_class',
 'waterpoint_type_group',
'longitude',
'latitude']]

y = df_mode['status_group']


X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42, stratify=y)

In [ ]:
numeric_features = ['longitude', 'latitude']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = [
 'source_type',
 'region',
 'district_code',
 'public_meeting',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'payment_type',
 'quantity_group',
 'source',
 'source_class',
 'waterpoint_type_group']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf_3 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(random_state = 42, class_weight= 'balanced', n_jobs = -1))])


clf_3.fit(X_train, y_train)


In [ ]:
clf_3.score(X_train, y_train)

In [ ]:
clf_3.score(X_test,y_test)

In [ ]:
yhat = clf_3.predict(X_test)

In [ ]:
recall_score(y_test, yhat, average='macro')

In [ ]:
cross_val_score(clf_3, X_test, y_test, scoring = 'recall_macro', cv = 5, n_jobs =-1)

In [ ]:
plot_confusion_matrix(clf_3, X_test, y_test)
plt.grid(None)
plt.savefig('saved_objects/final_model_confusion');

In [ ]:
cross_val_score(final_model, X_test, y_test, cv=5, n_jobs=-1, scoring='recall_macro')

In [ ]:
final_model_yhat = final_model.predict(X_test)
recall_score(y_test, final_model_yhat, average = 'macro')

In [ ]:
rfc_feat_import.score(X_mode_train_enc, y_train_cat)

In [ ]:
rfc_feat_import.score(ohe_X_test_enc, y_test_cat)

In [ ]:
asdfsda

In [ ]:
# cross_val_score(grid_search, X_train, y_train, cv=5, n_jobs=-1, scoring = 'recall_macro')
print('array([0.66330536, 0.64235777, 0.65301646, 0.63928187, 0.63296235])')

In [ ]:
feat_import_cat = grid_search.best_estimator_['preprocessor'].transformers_[0][1].get_feature_names(input_features = cat_features)
feat_import_num = grid_search.best_estimator_

In [ ]:
# from sklearn.preprocessing import PolynomialFeatures

# poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
# X_poly_train = pd.DataFrame(poly.fit_transform(X_train), columns=poly.get_feature_names(features.columns))
# X_poly_test = pd.DataFrame(poly.transform(X_test), columns=poly.get_feature_names(features.columns))
# X_poly_train.head()

In [ ]:
# from sklearn.feature_selection import VarianceThreshold

# threshold_ranges = np.linspace(0, 2, num=6)

# for thresh in threshold_ranges:
#     print(thresh)
#     selector = VarianceThreshold(thresh)
#     reduced_feature_train = selector.fit_transform(X_train)
#     reduced_feature_test = selector.transform(X_test)
#     lr = RandomForestClassifier()
#     lr.fit(reduced_feature_train, y_train)
#     run_model(lr, reduced_feature_train, reduced_feature_test, y_train, y_test)
#     print('--------------------------------------------------------------------')

In [ ]:
xgb = XGBClassifier(random_state = 42, objective = 'multi:softmax')

In [ ]:
cat_features = ['region_code', 'source_type', 'basin', 'region', 'district_code',\
                 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type',\
                 'extraction_type_group', 'extraction_type_class','management',\
                 'management_group', 'payment', 'payment_type', 'water_quality',\
                 'quantity', 'quantity_group', 'source',\
                 'source_class', 'waterpoint_type', 'waterpoint_type_group',\
                 'longitude', 'latitude']

categorical_transformer = OneHotEncoder(handle_unknown = 'ignore')

preprocessor = ColumnTransformer([('cat', categorical_transformer, cat_features)])

xgb = Pipeline([('preprocessor', preprocessor), 
               ('classifier', XGBClassifier(random_state = 42, objective = 'multi:softmax'))])

xgb.fit(X_train, y_train)

In [ ]:
xgb.score(X_train, y_train)

In [ ]:
xgb.score(X_test, y_test)

In [ ]:
y_hat = xgb.predict(X_test)

In [ ]:
recall_score(y_test, y_hat, average = 'macro')

In [ ]:
param_grid = {
 'classifier__max_depth': [3,10, None],
 
 'classifier__n_estimators': [100, 250, 500],
 'classifier__n_jobs': [-1],
 'classifier__learning_rate': [0.1,.01,.001],
 'classifier__booster': ['gbtree', 'gblinear', 'dart']
}

grid_search_xgb = RandomizedSearchCV(xgb, param_distributions=param_grid, scoring = 'recall_macro', n_iter = 60, n_jobs=-1, cv = 3, return_train_score=True)

grid_search_xgb.fit(X_train, y_train)

In [ ]:
grid_search_xgb.best_estimator_

In [ ]:
grid_search_xgb.best_params_

In [ ]:
grid_search_xgb.score(X_train, y_train)

In [ ]:
y_hat = grid_search_xgb.predict(X_test)

In [ ]:
recall_score(y_test, y_hat, average = 'macro')

In [ ]:
precision_score(y_test, y_hat, average = 'macro')

In [ ]:
accuracy_score(y_test, y_hat)

In [ ]:
plot_confusion_matrix(grid_search_xgb, X_test, y_test)
plt.grid(None);